In [1]:
import ctypes
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

class DacQueue:
    '''
    A queue for patterns sent to the dac. 
    Performs smart stitching between patterns.
    '''
    def __init__(self):
        # Dac object for this queue
        self.dac = Dac()
        # The last position of the last pattern (x,y)
        self.last_pos = (0,0)
        # Sample rate of the DAC
        self.dac_rate = 55000
        # Debugging dac rate
        #self.debug_rate = 550
        self.debug_rate = 150
        
    
    def submit(self, pat_pos, pat_col, angular_density=100, debug=False):
        '''
        Submits a new pattern to the dac, transitioning smoothly from the last one.
        Angular density describes how many points per radian should be used to transition
        '''
        # Make the transition
        dist = np.sqrt(np.sum(np.power(self.last_pos-pat_pos[0,:], 2)))
        num_gap_points = int(dist*angular_density)

        gap_pos = np.linspace(self.last_pos, pat_pos[0,:], num_gap_points, dtype=np.float)
        gap_col = np.zeros((num_gap_points, 3), dtype=np.float)
        if(debug):
            gap_col = np.ones_like(gap_col)/4

        # Prep the transition pattern
        gap_points, num_gap_points = self.prep_pattern(gap_pos, gap_col, gap=True)
        
        # Prep the new pattern
        pat_points, num_pat_points = self.prep_pattern(pat_pos, pat_col)
        
        # Send the transition pattern to the dac
        self.write_frames(gap_points, num_gap_points, do_not_loop=True, debug=debug)
        # Send the new pattern to the dac
        self.write_frames(pat_points, num_pat_points, do_not_loop=True, debug=debug)
        # Set the last_position
        self.last_pos = pat_pos[-1, :].copy()
        
    def prep_pattern(self, arr_pos, arr_col, gap=False):
        '''
        1) Scales and color corrects 
        2) Converts from unit space to DAC coordinates
        3) Produces a DAC compatible frame and displays it
        '''
        # Force pass-by-value
        arr_pos = arr_pos.copy()
        arr_col = arr_col.copy()
        # Scale the pattern 
        arr_pos = position_corection(arr_pos)
        # Format the position array (-1.0-1.0) -> int(0-4095)
        arr_pos = (((arr_pos+1)/2.0)*self.dac.xy_max).astype(np.int32)

        # Perform color correction if not gap points
        if(not gap):
            arr_col = color_correction(arr_col)
        # Format the color array (0-1.0) -> int(0-255)
        arr_col = (arr_col*255).astype(np.int32)
        # Fill a heliospoint arr with these values
        num_points = len(arr_pos)

        frameType = HeliosPoint * num_points
        points = frameType()
        # Fill the frame
        for idx in range(num_points):
            points[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                         int(arr_pos[idx, 1]), 
                                         int(arr_col[idx, 0]), 
                                         int(arr_col[idx, 1]),
                                         int(arr_col[idx, 2]),
                                         int(255))
        
        return points, num_points
            
    def write_frames(self, points, num_points, do_not_loop=False, start_immediately=False, debug=False):
        if(num_points < 1):
            return
        # Write the values out 
        status_attempts = 0
        max_attempts = np.inf
        # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
        while(status_attempts < max_attempts and self.dac.HeliosLib.GetStatus(0) != 1):
            status_attempts += 1
            
        # Create flags 
        flags = do_not_loop << 1 | start_immediately << 0
        # Send to DAC
        frame_rate = self.debug_rate if debug else self.dac_rate
        self.dac.HeliosLib.WriteFrame(0, frame_rate, flags, ctypes.pointer(points), num_points)

def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    non_zero = arr_color > 0
    # Scale to the lower cutoff (R, G, B)
    lower_cutoff = np.array([0.25, 0.09, 0.09])
    arr_color = arr_color*(1-lower_cutoff) + lower_cutoff
    
    # Cutoff any zeros to ensure real black
    arr_color *= non_zero
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
def make_circular(arr_pos, arr_color):
    return np.concatenate([arr_pos, arr_pos[::-1, :]]), np.concatenate([arr_color, arr_color[::-1, :]])

def connect_in_space(arr_pos_list, angular_density=100, wait_per=None):
    '''
    Given a list of position arrays, produce a list of array positions that are connected
    '''
    new_arr_pos_list = []
    for i in range(len(arr_pos_list)-1):
        # Get start/end positions for this gap
        pos_start = arr_pos_list[i][-1, :].copy()
        pos_end = arr_pos_list[i+1][ 0, :].copy()
        
        # Make the transition
        dist = np.sqrt(np.sum(np.power(pos_start-pos_end, 2)))
        num_gap_points = int(dist*angular_density)
        gap_pos = np.linspace(pos_start, pos_end, num_gap_points, dtype=np.float)
        
        # If we have a wait period, add it to the end of the gap positions
        if(wait_per):
            rep_bef = np.repeat(pos_start[np.newaxis, :], wait_per, axis=0)
            rep_aft = np.repeat(pos_end[np.newaxis, :], wait_per, axis=0)
            gap_pos = np.concatenate([rep_bef, gap_pos, rep_aft])
            
        
        new_arr_pos_list.append(arr_pos_list[i])
        new_arr_pos_list.append(gap_pos.copy())
    new_arr_pos_list.append(arr_pos_list[-1])
    
    return new_arr_pos_list

def fixed_interp(points, n_interp_points):
    '''
    Expands a list of points to a position array where those 
    points are the anchors in a smooth trajectory. 
    There is a fixed number of interp points between 
    the given points. 
    Inputs: 
        points - (N,2) - list of anchor points
        n_interp_points - number of points between anchors
    '''
    interped_pos_arr = []
    for i in range(len(points)-1):
        p1 = points[i]
        p2 = points[i+1]
        gap_pos = np.linspace(p1, p2, n_interp_points)
        interped_pos_arr += [p1]
        interped_pos_arr += [gap_pos]
    interped_pos_arr += [p2]
    
    # Convert them into one array 
    interped_pos_arr = np.concatenate(interped_pos_arr, axis=0)
    return interped_pos_arr

In [2]:
queue = DacQueue()

Found  1 Helios DACs


In [3]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


In [6]:
# CIELAB COLOR SPACE
start = -127
stop = 128
num_points = stop - start + 1

l = np.linspace(0, 100, num_points)
a = np.linspace(start, stop, num_points)
b = np.linspace(start, stop, num_points)
l, a, b = np.meshgrid(l, a, b, indexing='ij')
lab = np.stack([l, a, b], axis=-1)

# Convert LAB coords to RGB
rgb = lab2rgb(lab)

for lum in np.concatenate([np.arange(num_points), np.arange(num_points)[::-1]]):
    frame = rgb[lum, :, :, :]
    print('Lum', lum, np.min(frame), np.max(frame), np.mean(frame))
    cv2.imshow('frame', cv2.resize(frame, None, fx=2, fy=2))
    cv2.waitKey(10)


Lum 0 0.0 0.7578495556207594 0.12083769660173072
Lum 1 0.0 0.7622663711612824 0.12247870160369327
Lum 2 0.0 0.7666891448475324 0.12413461717475816
Lum 3 0.0 0.7711178446141438 0.12580463333765346
Lum 4 0.0 0.7755524388315317 0.1274885957734014
Lum 5 0.0 0.7799928962971352 0.12918604776259612
Lum 6 0.0 0.784439186226881 0.13089654546384583
Lum 7 0.0 0.788891278246857 0.1326201444985962
Lum 8 0.0 0.79334914238519 0.13435698971394086
Lum 9 0.0 0.797812749064126 0.1361062645855174
Lum 10 0.0 0.8022820690923003 0.13786736705121527
Lum 11 0.0 0.8067570736571987 0.13963994279565686
Lum 12 0.0 0.8112377343178024 0.14142274721982262
Lum 13 0.0 0.8157240229974071 0.14321417993348717
Lum 14 0.0 0.8202159119766196 0.14501308917717634
Lum 15 0.0 0.8247133738865196 0.14681843490079463
Lum 16 0.0 0.8292163817019864 0.1486287172382809
Lum 17 0.0 0.8337249087351839 0.15044317566919024
Lum 18 0.0 0.8382389286292014 0.1522612090219412
Lum 19 0.0 0.8427584153518429 0.15408302015605163
Lum 20 0.0 0.8472833

Lum 215 0.0 1.0 0.6573520014122124
Lum 216 0.0 1.0 0.6599728504523829
Lum 217 0.0 1.0 0.6625652275099128
Lum 218 0.0 1.0 0.6651300760185976
Lum 219 0.0 1.0 0.6676709088207765
Lum 220 0.0 1.0 0.6701901448955039
Lum 221 0.0 1.0 0.6726847516542303
Lum 222 0.0 1.0 0.6751534227133323
Lum 223 0.0 1.0 0.677606854926707
Lum 224 0.0 1.0 0.6800356050640067
Lum 225 0.0 1.0 0.6824428514252564
Lum 226 0.0 1.0 0.6848304643299913
Lum 227 0.0 1.0 0.6872015084921917
Lum 228 0.0 1.0 0.689549108054364
Lum 229 0.0 1.0 0.6918773178980269
Lum 230 0.0 1.0 0.694191279042891
Lum 231 0.0 1.0 0.6964841208037412
Lum 232 0.0 1.0 0.6987567470238113
Lum 233 0.0 1.0 0.701014121133828
Lum 234 0.0 1.0 0.7032550130524163
Lum 235 0.0 1.0 0.7054739615758021
Lum 236 0.0 1.0 0.707677064066433
Lum 237 0.0 1.0 0.7098645984022913
Lum 238 0.0 1.0 0.712033419106426
Lum 239 0.0 1.0 0.7141806057202986
Lum 240 0.0 1.0 0.7163160604266742
Lum 241 0.0 1.0 0.7184304451523246
Lum 242 0.0 1.0 0.7205247135717016
Lum 243 0.0 1.0 0.72260032

KeyboardInterrupt: 

In [17]:
# Sampling the CIELAB Space
while(True):
    # Choose a Lum
    lum_idx = int(num_points*np.random.rand(1))
    # Choose an A
    a_idx = int(num_points*np.random.rand(1))
    # Choose a B 
    b_idx = int(num_points*np.random.rand(1))

    point_rgb = rgb[lum_idx, a_idx, b_idx, :]
    
    # THIS IS NOT A PROPER WAY TO FIND THE COLOR'S COMPLEMENT! 
    # DO THIS IN LAB SPACE FIRST AND THEN CONVERT TO RGB!!!!!!!!!!!
    point_rgb_opp = rgb[lum_idx, -a_idx, -b_idx, :]

    frame = np.zeros((400, 200, 3))
    frame[0:200, :, :] = point_rgb
    frame[200:400, :, :] = point_rgb_opp
    cv2.imshow('point', frame)
    cv2.waitKey(0)


[0.90829142 0.         0.        ] [0.         0.45728094 0.91379411]
[1.         0.66301387 0.72266678] [0.         0.91249215 0.8988664 ]
[1.         0.         0.37707337] [0.         0.6359849  0.62229459]
[1.         0.         0.69411508] [0.        0.7640417 0.542856 ]
[0.71135554 0.04808678 0.66068684] [0.         0.4848053  0.07054322]
[0.53128221 0.51603441 1.        ] [0.37313371 0.70515003 0.        ]
[0.         0.30537227 0.08484186] [0.48533755 0.         0.37924545]
[0.51478992 0.27266104 1.        ] [0.         0.59848146 0.        ]
[0.03762454 0.22328169 0.        ] [0.         0.11208871 0.57494201]
[0. 1. 1.] [1.         0.57422148 0.63995909]
[1.         0.26581982 0.85758246] [0.         0.77282376 0.40932939]
[0.         0.68181382 1.        ] [0.78139538 0.64444749 0.        ]
[0.         0.39190984 0.7841724 ] [0.8155816 0.        0.       ]
[1.         0.74944537 1.        ] [0.14860826 1.         0.74226396]
[0.         0.42646086 0.97192933] [0.52575637 0.3

[0.79583011 0.40050401 1.        ] [0.10913785 0.67143206 0.        ]
[0.         0.12316081 0.37500585] [0.17905203 0.08286334 0.        ]
[0.         0.20042725 0.44642758] [0.31358792 0.06936005 0.        ]
[0.55082551 0.         0.        ] [0.         0.30326515 0.76843867]
[0.         0.10453499 0.23821629] [0.18950929 0.01938576 0.        ]
[0.         0.51473536 0.78017361] [0.7475319  0.26702112 0.01080224]
[0.         0.06153949 0.        ] [0.         0.0117521  0.44092105]
[0.706727   0.         0.68397749] [0.         0.43468066 0.        ]
[0.        1.        0.4283938] [1.         0.55434326 1.        ]
[0.         0.18164873 0.53676543] [0.22548929 0.13035694 0.        ]
[0.         0.4468721  0.10729206] [0.79269841 0.         0.55242428]
[0.         0.18239522 0.38534081] [0.32706275 0.         0.        ]
[0.88140925 0.85820008 0.56125714] [0.7796023  0.83231088 1.        ]
[0.60783418 0.34044091 0.48174572] [0.18305361 0.47701318 0.35693836]
[1.         0.46632576 

[0.         0.11873783 0.22423118] [0.25070807 0.         0.        ]


KeyboardInterrupt: 

In [ ]:
# Interpolated ellipses
T = 750
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

min_vals = np.array([0.8, 0.0, 1.0, 0.0, 5.00, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
max_vals = np.array([1.0, 0.4, 2.0, 4.0, 30.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
pat_point_old = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals


T_trans = 150

while(True):
    pat_point = max_vals*np.random.rand(len(max_vals))
    ell_interp = np.linspace(pat_point_old, pat_point, T_trans)
    for (scale_x, scale_y, freq_x, freq_y, f_s, phase, sintriang, r1, g1, b1, r2, g2, b2) in ell_interp:
        arr_pos, arr_col = ellipser(scale_x=scale_x, 
                                    scale_y=scale_y, 
                                    f_x=freq_x, 
                                    f_y=freq_y, 
                                    f_s=f_s, 
                                    phase_s=phase,
                                    sintriang=sintriang, 
                                    colors=[(r1,g1,b1), (r2,g2,b2)],
                                    T=T)
        
        queue.submit(arr_pos, arr_col)
        
    pat_point_old = pat_point